In [65]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf

spark = SparkSession.builder.\
            master("local").\
            appName("amzone").\
            config("packages",
        "databricks:spark-deep-learning:0.1.0-spark2.1-s_2.11").\
            enableHiveSupport().\
            getOrCreate()
sc = spark.sparkContext
sc.PACKAGE_EXTENSIONS=("databricks:spark-deep-learning:0.1.0-spark2.1-s_2.11")

In [64]:
sc.stop()
spark.stop()

In [66]:
from sparkdl import readImages
from pyspark.sql.functions import lit
img_dir = "data/flower"
tulips_df = readImages(img_dir + "/sunflower").withColumn("label", lit(1))
daisy_df = readImages(img_dir + "/daisy").withColumn("label", lit(0))
tulips_train, tulips_test = tulips_df.randomSplit([0.6, 0.4])
daisy_train, daisy_test = daisy_df.randomSplit([0.6, 0.4])
train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)
# train_df.show()

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print "Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label")))

+--------------------+--------------------+-----+
|            filePath|               image|label|
+--------------------+--------------------+-----+
|hdfs://master1:90...|[RGB,332,500,3,[B...|    1|
|hdfs://master1:90...|[RGB,212,320,3,[B...|    1|
|hdfs://master1:90...|[RGB,274,320,3,[B...|    1|
|hdfs://master1:90...|[RGB,332,500,3,[B...|    1|
|hdfs://master1:90...|[RGB,332,500,3,[B...|    1|
|hdfs://master1:90...|[RGB,282,320,3,[B...|    1|
|hdfs://master1:90...|[RGB,333,500,3,[B...|    1|
|hdfs://master1:90...|[RGB,336,500,3,[B...|    1|
|hdfs://master1:90...|[RGB,240,320,3,[B...|    1|
|hdfs://master1:90...|[RGB,243,320,3,[B...|    0|
|hdfs://master1:90...|[RGB,333,500,3,[B...|    0|
|hdfs://master1:90...|[RGB,240,240,3,[B...|    0|
|hdfs://master1:90...|[RGB,240,320,3,[B...|    0|
|hdfs://master1:90...|[RGB,240,240,3,[B...|    0|
|hdfs://master1:90...|[RGB,335,500,3,[B...|    0|
|hdfs://master1:90...|[RGB,225,320,3,[B...|    0|
|hdfs://master1:90...|[RGB,333,500,3,[B...|    0|


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])

model = p.fit(train_images_df)    # train_images_df is a dataset of images (SpImage) and labels

# Inspect training error
df = model.transform(train_images_df.limit(10)).select("image", "probability",  "uri", "label")
predictionAndLabels = df.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print "Training set accuracy = " + str(evaluator.evaluate(predictionAndLabels))

In [41]:
# import matplotlib.pyplot as plt
# from PIL import Image
# mode = image_df.collect()[0][1][0]
# size = image_df.collect()[0][1][1:3]
# color = image_df.collect()[0][1][4]
# im = Image.new(mode,size,color)
# # image_df.collect()[0][1][4]
# plt.imshow(im)
# # len()
image_df.collect()[0]

Row(filePath=u'hdfs://master1:9002/user/yanbin/data/mnist/0.110.jpg', image=None)

In [37]:
from PIL import Image 
import numpy as np
im = Image.open("/home/yanbin/data/mnist/0.998.jpg") 
print im.size,im.format,im.mode

(28, 28) JPEG L
